In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.0 MB/s eta 0:00:00


In [ ]:
# Imports
from google.colab import drive
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
import gradio as gr
import traceback



In [ ]:
# Mount Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import zipfile
import os

# Replace with the name of your uploaded ZIP file
zip_file_name = "/content/drive/MyDrive/BRAIN_DATA.zip"

# Extract to a folder
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall("brain_tumor_data")

print("Extraction complete.")


Extraction complete.


In [ ]:
import os

# Path to the dataset directory
dataset_path = '/content/brain_tumor_data/brain_tumor_dataset'  # Change this to your dataset path

# List subdirectories (e.g., yes, no)
classes = os.listdir(dataset_path)

# Loop through each class folder and count images
for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    if os.path.isdir(cls_path):
        image_count = len([f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))])
        print(f"{cls}: {image_count} images")


yes: 270 images
no: 203 images


In [ ]:
# Image parameters
img_height, img_width = 224, 224
batch_size = 32

# Data augmentation + normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)



Found 379 images belonging to 2 classes.
Found 94 images belonging to 2 classes.


In [ ]:
# Base model (MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_height, img_width, 3)))
base_model.trainable = False  # Freeze base

# Custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



<ipython-input-7-524d09eb6978>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_height, img_width, 3)))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=30
)

# Save the model
model.save("tumor_classification_model.h5")

# Save the training history
import pickle
with open("training_history.pkl", "wb") as f:
    pickle.dump(history.history, f)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


11/11 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.5987 - loss: 0.7431 - val_accuracy: 0.6562 - val_loss: 0.6130
Epoch 2/30
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6875 - loss: 0.6068

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.6875 - loss: 0.6068 - val_accuracy: 0.6719 - val_loss: 0.5996
Epoch 3/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 675ms/step - accuracy: 0.6351 - loss: 0.6519 - val_accuracy: 0.7500 - val_loss: 0.5260
Epoch 4/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step - accuracy: 0.7500 - loss: 0.6523 - val_accuracy: 0.6875 - val_loss: 0.5531
Epoch 5/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 451ms/step - accuracy: 0.6543 - loss: 0.6491 - val_accuracy: 0.8438 - val_loss: 0.5015
Epoch 6/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.7812 - loss: 0.4373 - val_accuracy: 0.7812 - val_loss: 0.5216
Epoch 7/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 553ms/step - accuracy: 0.7207 - loss: 0.5223 - val_accuracy: 0.9062 - val_loss: 0.4479
Epoch 8/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.6667 - loss: 0.5398 - val_accuracy: 0.8906 - val_loss: 0.4361
Epoch 9/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 636ms/step - accuracy: 0.7587 - loss: 0.4747 - val_accuracy: 0.9375 - val_

In [ ]:
# Generate predictions on validation data to get confusion matrix
val_preds = []
val_labels = []

# Reset generator before use
validation_generator.reset()
for i in range(len(validation_generator)):
    x_batch, y_batch = validation_generator[i]
    preds = model.predict(x_batch)
    val_preds.extend(preds.ravel() >= 0.5)  # Threshold 0.5
    val_labels.extend(y_batch)

# Confusion matrix
cm = confusion_matrix(val_labels, val_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Tumor", "Tumor"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png")
plt.close()

# Validation accuracy
val_acc = accuracy_score(val_labels, val_preds)
precision = precision_score(val_labels, val_preds)


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(32, 224, 224, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


In [ ]:
import gradio as gr
import numpy as np
import traceback
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import pickle

# Load model and history
model = load_model("tumor_classification_model.h5")

with open("training_history.pkl", "rb") as f:
    history = pickle.load(f)  # This is now a dictionary

# Define image size expected by the model
img_height, img_width = 224, 224  # Change if your model expects different size

# Preprocess function
def load_and_preprocess_image(img):
    img = img.resize((img_height, img_width))
    img_array = np.array(img) / 255.0
    if img_array.ndim == 2:  # grayscale
        img_array = np.expand_dims(img_array, axis=-1)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Prediction only
def predict_only(image):
    try:
        if image is None:
            raise ValueError("No image provided.")

        img_array = load_and_preprocess_image(image)
        prediction = model.predict(img_array)
        pred_label = 'Tumor' if prediction[0][0] >= 0.5 else 'No Tumor'

        # Accuracy graph
        plt.plot(history['accuracy'], label='Train Accuracy')
        plt.plot(history['val_accuracy'], label='Val Accuracy')
        plt.title("Accuracy")
        plt.legend()
        plt.savefig("accuracy_graph.png")
        plt.close()

        # Loss graph
        plt.plot(history['loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Val Loss')
        plt.title("Loss")
        plt.legend()
        plt.savefig("loss_graph.png")
        plt.close()

        # Return everything including confusion matrix image and val accuracy value
        return (
        pred_label,
        f"Validation Accuracy: {val_acc:.4f}\nPrecision: {precision:.4f}",
        "accuracy_graph.png",
        "loss_graph.png",
        "confusion_matrix.png"
)


    except Exception as e:
        traceback.print_exc()
        return "Error", None, None, None, None


# Gradio UI
interface = gr.Interface(
    fn=predict_only,
    inputs=gr.Image(type='pil'),
    outputs=[
        gr.Textbox(label="Predicted Class"),
        gr.Textbox(label="Validation Accuracy"),
        gr.Image(label="Accuracy Graph"),
        gr.Image(label="Loss Graph"),
        gr.Image(label="Confusion Matrix")
    ]
)

interface.launch(debug=True)




It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://34c48ba3b7fc8865bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


Traceback (most recent call last):
  File "<ipython-input-10-a7641db1695e>", line 31, in predict_only
    raise ValueError("No image provided.")
ValueError: No image provided.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
